<a href="https://colab.research.google.com/github/alex2844/youtube-music/blob/master/ymusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
if not os.path.exists('/content/drive/My Drive/ymusic'):
  os.mkdir('/content/drive/My Drive/ymusic')
os.chdir('/content/drive/My Drive/ymusic')
! bash <(curl 'https://raw.githubusercontent.com/alex2844/youtube-music/master/install.sh' -s)
! python <(curl 'https://raw.githubusercontent.com/alex2844/youtube-music/master/ymusic.py' -s -N)